In [16]:
import pandas as pd
import math
import numpy as np
import math
from sklearn.cross_validation import train_test_split # to split the dataset for training and testing
from sklearn.preprocessing import normalize # to normalize a matrix
from sklearn.decomposition import PCA
from sklearn import datasets, linear_model, metrics
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDRegressor
from scipy import linalg
from sklearn import metrics
import random
import pyswarm

In [2]:
# getting the ratings data in pandas dataframe
ratings = pd.read_csv('../Yahoo_movies_multi-criteria/data_movies.txt', sep='\t',names=['user_id', 'criterion1', 'criterion2', 'criterion3', 'criterion4', 'overall', 'movie_id', 'num'])
print(max(ratings.overall)) # implies that ratings are on a scale of 13
# train, test = sample_split(ratings)
# print(pd.Series.sort_values(ratings.movie_id).unique())

13


In [3]:
print("users: ",ratings.user_id.drop_duplicates().count()) #number of users
print("movies: ",ratings.movie_id.drop_duplicates().count()) #number of movies

users:  6078
movies:  976


In [4]:
print("minimum movies rated by a user" ,min(ratings.groupby(["user_id"]).overall.count())) # minimum ratings given by any user
print("maximum movies rated by a user" ,max(ratings.groupby(["user_id"]).overall.count())) # maximum ratings given by any user
print("minimum ratings of a movie" ,min(ratings.groupby(["movie_id"]).overall.count())) # minimum ratings of a movie
print("maximum ratings of a movie" ,max(ratings.groupby(["movie_id"]).overall.count())) # maximum ratings of a movie

minimum movies rated by a user 5
maximum movies rated by a user 237
minimum ratings of a movie 1
maximum ratings of a movie 830


In [5]:
# function to crop the dataset such that ratings given by any user>=ru
def min_user_ratings_criteria(ratings, ru):
    # remove all users who have rated <= ru movies
    temp = ratings.groupby(['user_id']).overall.count().reset_index()
#     print(temp)
    users_to_be_removed = temp[temp.overall < ru].user_id.unique()
    print(len(users_to_be_removed))
    new_ratings = ratings[~ratings['user_id'].isin(users_to_be_removed)]
    print(len(new_ratings.user_id.unique()))
    return new_ratings
#     print(len(users_to_be_removed))
# min_ratings_criteria(10,1)

In [6]:
# function to crop the dataset such that every item is rated by atleast ri users
def min_item_ratings_criteria(ratings, ri):
    # remove all items with number of ratings <= ri
    temp = ratings.groupby(['movie_id']).overall.count().reset_index()
#     print(temp)
    items_to_be_removed = temp[temp.overall < ri].movie_id.unique()
    print(len(items_to_be_removed))
    new_ratings = ratings[~ratings['movie_id'].isin(items_to_be_removed)]
    print(len(new_ratings.movie_id.unique()))
    return new_ratings
# min_item_ratings_criteria(5)

In [14]:
# split the datasets into smaller datasets, such that every dataset contains 'size' number of users
def split_dataset(ratings, size):
        df_group = ratings.groupby(['user_id'])
#         print(type(df_group))
        
        df_group_list = list()
        # converting the group of dataframes into list of dataframe for random shuffling
        for index, item in df_group:
            df_group_list.append(item)
        # randomly shuffling the list
        random.shuffle(df_group_list)
        no_of_datasets = int(len(df_group)/size);
#         print(no_of_datasets)
        list_datasets = list() # list containing all the small datasets
        temp = pd.DataFrame()
        count=0
        flag=0
        datasets_generated = 0 # datasets that have been generated till now
        for item in df_group_list:
            count+=1
            temp = temp.append(item)
            if count==size and datasets_generated < no_of_datasets-1:
                list_datasets.append(temp)
                print("size = ",temp.shape[0])
#                 print(temp.user_id.unique())
                temp = pd.DataFrame()
                datasets_generated += 1
                count=0
        list_datasets.append(temp) # appending the last dataset
        print("size = ",temp.shape[0])
        return list_datasets
    
# testing above function
# temp = split_dataset(ratings, 1000)
# for item in temp:
#     print(len(item.user_id.unique()))

In [9]:
# Spliting a sample into groups userwise
def sample_split(dataFrame):
    df_group = dataFrame.groupby('user_id')
    train = pd.DataFrame()
    test = pd.DataFrame()
    
    for key, item in df_group:
        train, test = split_train_test(item, train, test)
    
    return train, test

In [11]:
# spliting sample in 70% training data and 30% testing data
def split_train_test(dataFrame, train, test):
    temp_train, temp_test = train_test_split(dataFrame, test_size = 0.3, random_state=1212)# in this our main data is split into train and test
    # the attribute test_size=0.3 splits the data into 70% and 30% ratio. train=70% and test=30%
    train = train.append(temp_train)
    test = test.append(temp_test)
    
    return train, test

In [33]:
# remove movies with <5 ratings
rating_dash = min_item_ratings_criteria(ratings, 5)
# remove users who have rated <10 items
rating_dash = min_user_ratings_criteria(rating_dash, 10)
# print(len(rating_dash.movie_id.unique()))
# list conatining all the datasets
list_datasets = split_dataset(rating_dash,1000)
print(rating_dash.shape)

69
907
4371
1707
size =  34597
(34597, 8)


In [13]:
# splitting into train and test data
train = list()
test = list()
num_of_datasets = len(list_datasets)
print(num_of_datasets)
for i in range(num_of_datasets):
    print(i)
    t1, t2 = sample_split(list_datasets[i])
    train.append(t1)
    test.append(t2)

6
0
1
2
3
4
5


# Multi-criteria user-user collaborative filtering

In [15]:
# function to find similarity between two users based on Manhattan distance
def manhattan_similarity(df, user1, user2):
    s = pd.merge(df[df['user_id']==user1], df[df['user_id']==user2], how="inner", on="movie_id")
#     print(s)
    sum1 = 0
    similarity=0
    for index, row in s.iterrows():
        sum1 += abs(row.criterion1_x - row.criterion1_y) + \
                abs(row.criterion2_x - row.criterion2_y) + \
                abs(row.criterion3_x - row.criterion3_y) + \
                abs(row.criterion4_x - row.criterion4_y) + \
                abs(row.overall_x - row.overall_y)
        distance = sum1/s.shape[0]
        similarity = 1/(1+distance)
    return similarity
        
# manhattan_similarity(train, 1, 3)

In [16]:
# function to find simialrity between two users based on Euclidean distance
def euclidean_similarity(df, user1, user2):
    s = pd.merge(df[df['user_id']==user1], df[df['user_id']==user2], how="inner", on="movie_id")
#     print(s)
    sum1 = 0
    similarity=0
    for index, row in s.iterrows():
        sum1 += math.sqrt((row.criterion1_x - row.criterion1_y)**2 + \
                (row.criterion2_x - row.criterion2_y)**2 + \
                (row.criterion3_x - row.criterion3_y)**2 + \
                (row.criterion4_x - row.criterion4_y)**2) + \
                (row.overall_x - row.overall_y)**2
        distance = sum1/s.shape[0]
        similarity = 1/(1+distance)
    return similarity
# euclidean_similarity(train, 1, 3)

In [17]:
# function to find simialrity between two users based on Chebyshev distance
def chebyshev_similarity(df, user1, user2):
    s = pd.merge(df[df['user_id']==user1], df[df['user_id']==user2], how="inner", on="movie_id")
#     print(s)
    sum1 = 0
    similarity=0
    for index, row in s.iterrows():
        sum1 += max(abs(row.criterion1_x - row.criterion1_y), \
                abs(row.criterion2_x - row.criterion2_y), \
                abs(row.criterion3_x - row.criterion3_y), \
                abs(row.criterion4_x - row.criterion4_y), \
                abs(row.overall_x - row.overall_y))
        distance = sum1/s.shape[0]
        similarity = 1/(1+distance)
    return similarity
        
# chebyshev_similarity(train, 1, 3)

In [18]:
# function to predict the rating given by user to item
# neighbours = -1 implies that consider similarity with all possible users
def predict(df, user, item, similarity, neighbours = -1): # df is the train dataset
    neighbours_data_list = list()
    for v in df.user_id.unique():
        if(v==user): # not including the user itself
            continue
        temp = df[df['user_id']==v]
        temp = temp[temp['movie_id']==item]
        if(temp.empty): # user 'v' has not rated the item
            continue
        else:
            rate = temp.iloc[0].overall
        sim = similarity(df, user, v) # find appropriate similarity measure between the two users
        neighbours_data_list.append((sim, rate))
        
    # sort the neighbours_data_list in descending order based on rate
    neighbours_data_list.sort(reverse=True)
    
    # crop the list to the number of neighbours given in the argument
    if(neighbours!=-1):
        length = len(neighbours_data_list)
        neighbours_data_list = neighbours_data_list[:min(neighbours, length)]
    
    # predict the rating using collaborative filtering formula
    numerator = 0
    denominator = 0
    if(len(neighbours_data_list)==0):
        print("We cannot predict")
    for tup in neighbours_data_list:
        numerator += tup[0]*tup[1]
        denominator += tup[0]
    if(denominator==0):
        return 0
    predicted_rating = numerator/denominator
    return predicted_rating
        

In [19]:
# print(test[0].head(26))

In [20]:
# print(train)
print(predict(train[0], 1, 879, euclidean_similarity))
print(predict(train[0], 1, 879, manhattan_similarity))
print(predict(train[0], 1, 879, chebyshev_similarity))

0
0
0


In [19]:
# function to calculate the mean average error between two vectors
def mean_average_error(list1, list2):
    sum1=0
    for a,b in zip(list1,list2):
        if(b==0):            # to handle the unpredicted ratings
            continue
        sum1 += abs(a-b)
    error = sum1/len(list1)
    return error
mean_average_error([1,2,3],[10,20,30])

18.0

In [20]:
# function to replace entries in a list with zero/one based on some threshold
def replace_with_zero_one(list1, threshold):
    length = len(list1)
    rlist = list()
    i = 0
    while i < length:
        if(list1[i]==0):   # to handle the unpredicted ratings
            list1[i]=-1
            continue
        if(list1[i]>threshold):
            rlist.append(1)
        else:
            rlist.append(0)
        i += 1
    return rlist

In [21]:
# function to calculate the precision between two lists
# list2 is the predicted list
def precision(list1, list2, threshold): # values > threshold are considered as good values
#     list1 = replace_with_zero_one(list1, threshold)
#     list2 = replace_with_zero_one(list2, threshold)
    tp = 0 # true positives
    fp = 0 # false positives
    for a,b in zip(list1, list2):
        if(a==1 and b==1):
            tp += 1
        if(b==1 and a==0):
            fp+=1
    precision = tp/(tp + fp)
    return precision

# precision([20,5,14], [1,16,20], 13)

In [22]:
# function to calculate the recall between two lists
# list2 is the predicted list
def recall(list1, list2, threshold): # values > threshold are considered as good values
#     list1 = replace_with_zero_one(list1, threshold)
#     list2 = replace_with_zero_one(list2, threshold)
    tp = 0 # true positives
    fn = 0 # false negatives
    for a,b in zip(list1, list2):
        if(a==1 and b==1):
            tp += 1
        if(b==0 and a==1):
            fn += 1
#     print("tp = ",tp)
#     print("fn = ",fn)
    recall = tp/(tp + fn)
    return recall

# recall([20,15,14], [1,16,15], 13)

In [23]:
# function to calculate F1-score between two lists
def f1_score(list1, list2, threshold):
    p = precision(list1, list2, threshold)
    r = recall(list1, list2, threshold)
    f1 = (2*p*r)/(p+r)
    return f1

In [24]:
# predicting on our training dataset
predicted = list() # for storing the predicted ratings
# appending 7 empty lists
i=0
while i<len(list_datasets):
    predicted.append([])
    i+=1
unpredicted_count = list()
# print(predicted1)

In [27]:
# bulk testing on all datasets
for i in range(len(list_datasets)):
    cnt = 0
    for index, row in test[i].iterrows():
        predicted_rating = predict(train[i], row.user_id, row.movie_id,chebyshev_similarity)
#         print(predicted_rating)
        if(math.isnan(predicted_rating)):
            print("Cannot predict")
            continue
        if(predicted_rating==0):
            cnt += 1
        predicted[i].append(predicted_rating)
        print(predicted_rating)
    print("predicted %d",i)
    unpredicted_count.append(cnt)

6.0
0
9.34528605963
4.04347826087
11.25
We cannot predict
0
0
0
13.0
9.0
9.84158415842
10.0794775293
12.1232032854
11.9938650307
10.3560797529
12.2122356495
0
0
5.0
10.9189189189
10.5
10.2988854286
We cannot predict
0
10.0769230769
4.83495145631
11.2195121951
0
8.5
11.0324324324
7.90909090909
10.1956521739
1.0
9.10160275661
0
8.95625635809
11.4613402062
9.82075471698
11.8722891566
12.5613577023
11.0
10.0
11.4285714286
6.89156626506
8.95652173913
7.95652173913
9.42857142857
11.6071428571
11.2
5.1186440678
We cannot predict
0
11.132078505
9.203125
7.0487804878
9.05515451375
7.42857142857
7.0
11.887822226
6.76923076923
4.85714285714
11.0801409071
11.9465747519
8.1948376353
10.5135135135
10.0
0
5.36666666667
9.0
6.2
9.66666666667
10.4
8.2
7.0
13.0
4.875
0
12.0
12.1208791209
11.5591127155
0
4.0
7.74747474747
6.0
0
12.4412269004
11.1258709309
10.2
11.0
9.58536585366
11.6379444589
5.79560602623
8.175066313
10.4482315113
10.063469109
8.35709220283
9.52525914504
11.8188861219
10.3054791546
9.24

KeyboardInterrupt: 

In [28]:
# predicting on our training dataset1
print("test-size = ", test[1].shape[0])
predicted[1] = list() # for storing the predicted ratings
i=0
for index, row in test[1].iterrows():
    predicted_rating = predict(train[1], row.user_id, row.movie_id,euclidean_similarity, 20)
    print(i)
    i+=1
    if(math.isnan(predicted_rating) or predicted_rating==0):
        print("Continuing")
        continue
    predicted[1].append(predicted_rating)
print(predicted[1])

test-size =  3646
0
1
2
3
4
5
6
Continuing
7
8
9


KeyboardInterrupt: 

In [29]:
# print("unpredicted = ",unpredicted_count[1])
print(len(predicted[1]))
threshold = 7
ttest = test[1].overall
list1 = replace_with_zero_one(ttest.tolist(), threshold)
list2 = replace_with_zero_one(predicted[1], threshold)
# mean average error
print("mae = ",mean_average_error(ttest, predicted[1]))
# precision
print("precision = ",precision(list1, list2, threshold))
# recall
print("recall = ",recall(list1, list2, threshold))
# f1 score
print("f1-score = ",f1_score(list1, list2, threshold))

9
mae =  0.00686817891691
precision =  0.8571428571428571
recall =  0.75
f1-score =  0.7999999999999999


# Singular Value Decomposition (SVD)

In [30]:
# function to convert data into mxn matrix m -> number of users, n -> number of movies
# the matrix is normalized to take care of missing ratings
# after normalizations missing ratings can be safely assumed to be zero
# df dataframe should contain columns as 'user_id', 'movie_id', 'rate'
def convert_to_normalized_matrix(df, rating_column):
    no_of_movies = len(ratings.movie_id.unique())
    no_of_users = len(ratings.user_id.unique())
    matrix = np.zeros((no_of_users, no_of_movies))
    for index, row in df.iterrows():
        matrix[row.user_id-1][row.movie_id-1] = row[rating_column]
#     print("completed")
    return matrix,normalize_matrix(matrix)

# function to normalize a matrix by subtracting the average rating of the user
def normalize_matrix(matrix):
#     matrix[matrix == 0] = np.nan
#     matrix = np.ma.array(matrix, mask=np.isnan(matrix))

    # normalize the matrix
    normalized_matrix = np.zeros((len(matrix), len(matrix[0])))
    i=0
#     print(matrix)
    num_users = len(matrix)
    while i < num_users:
        baseline = np.sum(matrix[i])
        baseline /= np.count_nonzero(matrix[i])
        # subtract baseline value from all non-zero ratings
        normalized_matrix[i][:] = [x-baseline if x>0 else x for x in matrix[i]]
        i+=1
    return normalized_matrix
    
    
# g,h = convert_to_normalized_matrix(ratings[['user_id', 'movie_id', 'overall']])
# print(h)

In [31]:
# function to apply SVD on normalized user-item matrix
def svd(ratings, rating_column):
    original,X = convert_to_normalized_matrix(ratings[['user_id', 'movie_id', rating_column]], rating_column)
    print("matrix created, now decomposing it")
    u,s,v = np.linalg.svd(X) # Singular Value Decomposition
    return original,u,s,v

In [32]:
# function to crop the decomposed matrices to top-k latent features
def crop_matrices(u, s, v, k):
    # calculate average rating of the user according to 'matrix'
    cropped_u = u[:,:k] # crop matrix u
    cropped_s = s[:k] # crop array s
    cropped_v = v[:k,:] # crop matrix v
    return cropped_u, cropped_s, cropped_v
# crop_matrices(u,s,v,15)

In [33]:
# to predict the rating given by 'user' to 'item' based on SVD
# u, s, v are the decomposed matrices
# matrix is the parent matrix that is decomposed
def predict_svd(matrix, u, s, v, user, item):
    # calculate average rating of 'user' according to 'matrix'
    baseline = np.sum(matrix[user-1])
    baseline /= np.count_nonzero(matrix[user-1])
#     print("average -> ", baseline)
    p = u[user-1] # latent-features values of the user
    # s is the weight of latent-features
    q = np.transpose(v)[item-1]
    
    return baseline+sum(p*s*q)
    
# matrix,u,s,v = svd(ratings,'overall')
# print("completed")
# a,b,c = crop_matrices(u,s,v,15)
# print("completed")
# predict_svd(matrix,a,b,c,1,1)

In [34]:
# creating the decomposed matrices
matrix,u,s,v = svd(train[0], 'overall')

C:\Users\Garvit\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


matrix created, now decomposing it


In [35]:
# croppping the matrices to appropriate no. of latent-features
a,b,c = crop_matrices(u,s,v,15)

In [36]:
# testing the svd implementation
# fingers crossed
# predicting on our training dataset1
print("test-size = ", test[0].shape[0])
predicted[0] = list() # for storing the predicted ratings
i=0
for index, row in test[0].iterrows():
    predicted_rating = predict_svd(matrix,a,b,c,row.user_id, row.movie_id)
#     print(i)
    i+=1
    if(math.isnan(predicted_rating) or predicted_rating==0):
        print("Continuing")
        continue
    predicted[0].append(predicted_rating)
# print(predicted[0])

test-size =  3433


In [37]:
# creating empty lists for storing the predicted values
predicted = list() # for storing the predicted ratings
i=0
while i<len(list_datasets):
    predicted.append([])
    i+=1

In [38]:
# bulk testing on all datasets
for i in range(len(list_datasets)):
    matrix,u,s,v = svd(train[i], 'overall')
    a,b,c = crop_matrices(u,s,v,15)
    for index, row in test[i].iterrows():
        predicted_rating = predict_svd(matrix,a,b,c,row.user_id, row.movie_id)
        if(math.isnan(predicted_rating)):
            print("Cannot predict")
            continue
        if(predicted_rating==0):
            cnt += 1
        predicted[i].append(predicted_rating)
#         print(predicted_rating)
    print("predicted ",i)
#     unpredicted_count.append(cnt)

C:\Users\Garvit\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars


matrix created, now decomposing it
predicted  0
matrix created, now decomposing it
predicted  1
matrix created, now decomposing it
predicted  2
matrix created, now decomposing it
predicted  3
matrix created, now decomposing it
predicted  4
matrix created, now decomposing it
predicted  5


In [39]:
# Evaluating the model
print(len(predicted[5]))
threshold = 7
ttest = test[5].overall
list1 = replace_with_zero_one(ttest.tolist(), threshold)
list2 = replace_with_zero_one(predicted[5], threshold)
# mean average error
print("mae = ",mean_average_error(ttest, predicted[5]))
# precision
print("precision = ",precision(list1, list2, threshold))
# recall
print("recall = ",recall(list1, list2, threshold))
# f1 score
print("f1-score = ",f1_score(list1, list2, threshold))

3696
mae =  2.64854391853
precision =  0.7879243015920697
recall =  0.9258736321920226
f1-score =  0.8513469652710159


# Particle Swarm Optimization (PSO)
## (using the pyswarm library)

In [133]:
# defining the fitness funcion to be optimised by pso
# weights is the list of weights that are to be calculated by PSO
# 'user' is the user for which we want to predict the weights
user = pd.DataFrame()
def fitness_func(weights):
    w1 = weights[0]
    w2 = weights[1]
    w3 = weights[2]
    w4 = weights[3]
    ans = 0
    for index, row in user.iterrows():
        ans += (w1*row.criterion1 + w2*row.criterion2 + w3*row.criterion3 + w4*row.criterion4 - row.overall)**2
    return ans

In [156]:
# function for predicting the label according to our weights got using pso
def predict_pso(xopt, test):
    predicted_list = list()
    w1 = xopt[0]
    w2 = xopt[1]
    w3 = xopt[2]
    w4 = xopt[3]
    for index, row in test.iterrows():
        prediction = w1*row.criterion1 + w2*row.criterion2 + w3*row.criterion3 + w4*row.criterion4
        predicted_list.append(prediction)
    return predicted_list

In [175]:
# Anubhav's implementation of PSO
def pso(fitness_func, lbound, ubound, swarm_size=20, max_iter=40, dimensions=4, phip=0.5, phig=0.5, weight=0.017):
    x=[]            # positon of each particle
    x_pbest=[]      # best position of each particle 
    x_gbest=[]      # best global position of population
    v=[]            # velocity of each particle
    v_min=[]
    v_max=[]
    
    # assign initial random positions to the particles
    for i in range(swarm_size):
        x.append([])
        x_pbest.append([])
        for j in range(dimensions):
            x[i].append(random.uniform(lbound[j],ubound[j]))
            x_pbest[i].append(x[i][j])
            
    #calculate initial group best of the population
    for i in range(swarm_size):
        if i==0:
            x_gbest=x[i]
        elif fitness_func(x[i])<fitness_func(x_gbest):
            x_gbest=x[i]
            
            
    r1=random.uniform(0,1)
    r2=random.uniform(0,1)
    
    # calculate minimum and maximum boundaries of velocity vector
    for i in range(dimensions):
        v_min.append(-(ubound[i]-lbound[i])/10)
        v_max.append((ubound[i]-lbound[i])/10)
 
    # assign initial random velocities to the particles
    for i in range(swarm_size):
        v.append([])
        for j in range(dimensions):
            v[i].append(random.uniform(v_min[j],v_max[j]))
            
    for iter in range(max_iter):
        for i in range(swarm_size):
            for j in range(dimensions):
                # calculate new velocity for each particle
                v[i][j] = weight*(v[i][j]) + r1*phip*(x_pbest[i][j]-x[i][j]) + r2*phig*(x_gbest[j]-x[i][j])
                
#                 if v[i][j] > v_max[j]:
#                     v[i][j] = v_max[j]
                
#                 if v[i][j] < v_min[j]:
#                     v[i][j] = v_min[j]


                # Kumud's code
                #checking the limit of velocity, if goes beyond then sack it
                if v[i][j] > v_max:
                    vel[i][j] = (vel_max / abs(vel[i][j])*vel[i][j]
                                 
                                 
                
                # calculate new position for each particle
                x[i][j] = x[i][j] + v[i][j]
                
                if x[i][j] > ubound[j]:
                    x[i][j] = ubound[j]
                    
                if x[i][j] < lbound[j]:
                    x[i][j] = lbound[j]
            
            if fitness_func(x[i]) < fitness_func(x_pbest[i]):
                x_pbest[i] = x[i]
                
            if fitness_func(x[i]) < fitness_func(x_gbest):
                x_gbest = x[i]
                
    return x_gbest,fitness_func(x_gbest)

SyntaxError: invalid syntax (<ipython-input-175-9cc6ba821094>, line 61)

In [176]:
# applying the particle-swarm-optimization
# xopt = the optimal input values
# fopt = the optimal objective value
df = list_datasets[0]
user_list = df.groupby(['user_id'])
lower_bound = [0, 0, 0, 0]
upper_bound = [255, 255, 255, 255]
mae = 0
count = 0
for index, item in user_list:
    # split item in train and test data
    train, test = train_test_split(item, test_size = 0.3, random_state = 1601)
    user = train
    # library implemenatation
#     xopt, fopt = pyswarm.pso(fitness_func, lower_bound, upper_bound)
    # Anubhav's implementation
    xopt, fopt = pso(fitness_func, lower_bound, upper_bound)
    print(xopt)
    print(fopt)
    # predict the values on the test dataset
    predicted_list = predict_pso(xopt, test)
#     print("test shape ", test.shape[0])
    # calculate the mean average error
#     print("checking --> ",test.overall.shape[1])
#     print("debug = ",len(predicted_list))
#     print("debug = ", len(test.overall))
    error = metrics.mean_absolute_error(test.overall.tolist(), predicted_list)
    accuracy = metrics.r2_score(test.overall.tolist(), predicted_list)
    print("mae = ",error)
    mae += error
    count += 1
    if(count==20):
        break

[0, 0, 0, 0]
264
mae =  5.5
[0, 0, 0, 0]
306
mae =  12.5
[0, 63.90047939062116, 0, 16.663029575979127]
2749424.80397
mae =  901.719768288
[0, 0, 42.18954725523602, 0.6950620000336839]
459901.78912
mae =  148.181318394
[0, 0, 0, 0]
402
mae =  13.0
[0, 0, 0, 0]
171
mae =  12.5
[0, 0, 19.409921877282674, 0]
353437.001777
mae =  146.612708352
[0, 0, 0, 0]
492
mae =  12.0
[0, 0, 0, 0]
393
mae =  10.0
[0, 0, 0, 0]
1161
mae =  9.8
[0, 0, 0, 0]
603
mae =  8.5
[0, 0, 0, 0]
492
mae =  6.5
[0, 0, 0, 0]
351
mae =  5.66666666667
[0, 0, 0, 0]
371
mae =  5.5
[0, 0, 0, 0]
486
mae =  11.0
[0, 0, 0, 0]
460
mae =  11.0
[0, 0, 0, 0]
557
mae =  12.0


KeyboardInterrupt: 

In [1]:
print("mae = ",mae/count)

NameError: name 'mae' is not defined

# Linear Regression
## (using library provided by sklearn)

In [38]:
# applying linear regression on list_datasets[0]
# print(list_datasets[0])
for i in range(len(list_datasets)):
    df = list_datasets[i]
    df_group = df.groupby(['user_id'])
    count = 0
    avg_error = 0
#     print("mae = ",avg_error/len(list_datasets[i].user_id.unique()))
    for key,item in df_group:
        # splitting the item in train and test data
        train, test = train_test_split(item, test_size = 0.2, random_state=1212)
        train_X = train[['criterion1', 'criterion2', 'criterion3', 'criterion4']]
        train_y = train.overall
        test_X = test[['criterion1', 'criterion2', 'criterion3', 'criterion4']]
        test_y = test.overall
        # create linear regression object
        clf = linear_model.LinearRegression(normalize=True)
        # train the model using the training sets
        clf.fit(train_X, train_y)
        prediction = clf.predict(test_X)
        st=0
        for i in range(len(prediction)):
            print(prediction[i], test_y.tolist()[i])
            st = max(st, abs(prediction[i]-test_y.tolist()[i]))
        # checking the mean average error
#         score = clf.score(test_X, test_y)
        score = mean_average_error(prediction, test_y)
#         print(score)
        avg_error += score 
        if(score<0):
            count+=1
    print(avg_error)
    tt = avg_error/len(list_datasets[i].user_id.unique())
    print("mae = ",tt)
    print("negative = ", count)

5.67165777253 5
9.94522566345 10
6.78708612871 3
2.27641847685 1
11.0843044942 11
10.1268925826 11
7.48571277726 3
7.21321239861 6
8.70203149157 10
11.6689349164 12
12.6468328803 13
10.2224613837 10
11.6791673711 11
3.37436000815 1
9.77285870974 10
10.1631164284 9
12.972891742 13
12.972891742 13
11.2413237951 11
11.0471107808 11
11.8952067352 12
12.5901566181 12
12.5901566181 12
12.7944286172 13
11.2384800846 11
12.4266513387 13
12.8349513145 13
13.2354170757 13
12.9784525631 13
6.89586603872 5
11.461189604 11
12.4584157297 11
10.0464621353 10
12.4744712507 12
12.0219443471 12
8.71741743266 10
9.26030684056 11
11.7500914757 12
12.9494016297 13
10.284241676 9
12.9494016297 13
12.9494016297 13
12.9494016297 13
12.9494016297 13
12.9494016297 13
6.99688581684 7
2.03645597281 2
12.9494016297 13
6.99688581684 7
12.9494016297 13
7.98897178565 8
13.0679719297 13
12.9800623366 13
11.9579397581 11
9.12357954545 9
12.2215909091 12
11.6505681818 11
12.8421802636 13
11.9549340933 13
11.9904382869 1

7.66133412869 8
12.194659993 12
12.22279321 13
11.7657284768 10
10.8998344371 10
12.8592715232 12
14.3027295285 13
14.3027295285 13
10.300248139 10
11.3275434243 12
8.0532205841 8
13.2588371611 13
7.26337672916 8
12.6010298059 12
0.0864417965542 1
12.1611375474 12
12.480636979 12
10.9430451067 11
7.12200956938 7
14.1818181818 13
7.12041467305 6
10.424178273 12
13.0 13
13.0 13
8.4249180763 12
8.35615985895 6
-0.0712481051036 2
10.3946437595 10
9.42748863062 10
5.42672274345 10
12.056163054 12
9.16609511485 8
5.78990000721 2
9.77714323021 11
1.21345307122 3
3.07246440432 4
10.8897470384 9
8.94777905263 10
11.8607310313 12
6.09395582193 7
8.76257507784 10
7.97679505973 9
13.0900692841 13
8.51154734411 9
13.539516067 13
9.15544781944 11
11.9606103581 13
12.7363543419 12
0.910985729931 2
3.18241644228 1
6.0766789358 7
8.89549102748 8
10.3041126501 10
10.192955558 11
10.0710053078 10
10.3041126501 10
0.979520735757 6
6.87937611282 1
12.361880227 13
10.4464454301 11
9.35735198439 11
11.127602

6.39835626988 6
11.5405439748 12
11.5405439748 12
8.22832592699 5
10.2489863106 10
9.23136883419 11
9.26701817162 8
8.61760966306 11
11.0251112524 12
6.21010807374 13
7.33333333333 6
8.87777777778 9
9.4529129371 7
10.3567349088 11
10.4230185524 11
10.5368445504 10
11.425415802 11
8.96696503237 9
11.6596573209 13
8.46962616822 13
9.28115264798 12
12.9827586207 13
6.8599137931 6
0.774011803637 1
5.61774773982 1
11.5934719633 10
8.57171160905 10
11.4733460557 13
12.3197438788 12
10.8858901782 12
12.0652211555 13
10.2581685666 11
10.0606103628 10
5.47912737265 5
13.2114561028 13
7.07334047109 7
9.16113490364 4
8.04817987152 8
12.2162676981 12
7.87439340519 12
11.7903902013 12
12.0424856891 12
10.8775567929 11
6.89348533144 8
12.5132267247 12
2.97550353838 3
5.42496824533 1
2.97550353838 3
13.0 13
13.0 13
10.9070280271 11
2.23191292014 1
11.5253748407 12
8.78414839077 10
12.6429514194 12
12.9053022394 13
9.21760170254 10
10.4635328565 6
4.10938623534 7
11.6620952127 13
12.4327246743 9
8.508

11.9091321198 13
13.1347990871 13
12.9905005778 13
10.5301584779 12
9.16270120955 10
9.99085365854 10
10.9756097561 12
10.8841463415 13
7.92168944609 7
8.02667095522 5
5.9752574887 5
10.0825607151 11
-0.0855250686582 1
10.2321972223 9
3.49340470262 3
10.0806244968 10
10.2321972223 9
10.8404647221 10
5.90064852103 8
1.90225771497 2
11.7432084208 11
11.9163498705 12
10.4002057847 10
7.88802839569 4
7.86165354677 2
12.5102277967 13
9.83026083209 11
5.67711448776 5
10.9106907977 11
4.55599278627 5
13.1838416992 13
6.27188483314 5
10.4776716442 11
12.7339816962 13
5.67670844324 4
10.6036232189 10
11.3034402118 12
12.1151261785 11
6.38847416703 5
7.96157378963 8
8.30191702299 7
12.6450839329 11
13.0679456435 13
10.7889688249 11
11.0255795364 11
13.0679456435 13
12.6450839329 12
6.67854781754 5
10.67518626 10
11.0990516411 11
11.8453660557 12
10.7415832353 10
1.07129239243 1
4.18133168527 1
9.44698327789 10
10.0773029585 10
10.859694618 11
10.5472757531 11
10.6360031635 10
12.170327455 12
13.

13.0741363212 13
11.4924836601 11
12.7044817927 13
10.7049486461 12
13.8716119829 12
11.455064194 9
9.32876334423 9
5.3142103881 4
12.151933272 13
9.07114737318 9
11.835876438 12
12.151933272 12
12.151933272 13
12.4090909091 12
12.5606060606 13
9.33333333333 9
9.66666666667 10
9.0 9
11.6646826289 11
9.37935254104 9
9.55083260597 9
10.9884283954 12
7.70700398291 9
9.16972835482 8
11.2530237375 13
13.4308361314 8
4.43301040348 5
13.4308361314 9
5.8729689808 5
11.341211226 13
10.5243722304 11
13.0 13
8.17261121348 9
10.3127098736 11
8.88096233204 7
12.1254221492 11
11.0552326908 12
10.1993767865 12
10.6051875012 10
8.85628735401 8
9.50831215999 8
10.6051875012 10
10.5641858975 11
10.3972315436 11
12.4450503356 13
9.52726510067 6
11.166960156 12
7.54147334972 11
11.166960156 12
6.02840317947 5
11.1544693807 11
10.9725024209 11
10.6788452227 11
11.0209611039 11
7.24557679234 6
11.2205865922 11
8.60014732979 9
10.2852355856 11
13.0213523132 13
7.68042704626 11
12.3685481863 13
10.1353772236 

11.9877133106 12
12.2685267694 12
11.0089999258 12
7.49567510165 7
11.3843580919 1
13.3154729321 13
13.3154729321 13
11.3199639924 12
10.7308058616 12
12.4564757709 12
12.6539219894 12
3.19846371664 1
13.3154729321 13
11.602388781 12
11.6705508686 12
11.2605195395 11
8.43688490832 9
6.62139525173 6
11.5603379767 12
9.25354057588 10
7.92630805661 8
10.3041941624 10
11.25 10
8.75 10
13.0 13
8.54130938455 9
9.75817214445 7
9.97723557602 10
10.9750349043 11
9.20119693596 12
13.0 12
13.0 13
13.0 13
10.932552763 13
6.6123899808 10
9.88872163945 10
12.284815667 10
5.88493107548 5
7.10237754021 7
10.5425029174 11
13.3830254653 13
12.7658913741 12
11.9717003172 12
5.79037364592 6
10.4936650368 11
7.28577260816 7
8.08876727111 8
7.30845139471 7
10.2888299587 10
7.34169506879 8
12.6847733688 13
13.2359978999 13
10.3282426621 11
9.32247976292 7
13.2359978999 13
12.496200049 12
10.6160823731 10
9.10301668806 9
9.45667522465 10
9.4756097561 9
11.7654043646 11
9.57028241335 10
8.26326223775 8
11.9731

4.62943049736 4
11.7136530252 12
8.0863830756 10
8.44863176276 9
9.41674836032 9
5.90968509214 4
9.12167686285 11
9.22549640757 11
6.01284241563 6
11.7136530252 12
11.7136530252 12
7.91311261881 7
9.75 9
12.0 12
8.25 9
9.58973199141 10
9.24827547212 9
10.9557276942 11
13.0 13
13.0 13
13.0 13
13.0879860979 13
12.0188106704 12
13.0879860979 13
5.68083002791 6
13.0879860979 13
12.0188106704 12
13.0879860979 13
9.81324287985 10
8.81128438796 10
12.5237507533 13
13.0879860979 13
10.2674415813 11
12.9404007444 13
5.19322608001 6
12.4248149167 12
13.0228507506 13
10.8674113495 12
12.9404007444 13
12.3385489806 12
9.30098738322 9
6.68213300997 4
8.8768879617 8
7.47181084138 7
8.15318342789 7
9.43794461376 10
12.1561750637 13
9.33336551185 10
5.85905239947 7
7.49422758138 5
8.79370094933 8
1.53133741882 2
8.89307702602 10
4.42365402138 5
9.08084314031 9
6.34019595037 5
9.45888447153 9
8.23264429727 10
12.2978830877 13
8.3534124635 8
11.7320756422 12
13.5665850916 13
12.9620455012 13
-4.29178557

11.2218911332 10
11.0782718039 12
1.82234211159 1
-0.253787982266 3
6.2770755886 6
11.0765799257 11
13.3372986369 12
12.292936803 13
6.20536756126 5
10.7236872812 12
8.25075845974 10
9.72331606694 13
13.479048171 10
11.6526590006 12
9.0136536077 13
1.07799500673 1
12.5342689646 7
10.8312900792 10
13.1037344398 12
12.8571428571 13
12.8571428571 13
14.2857142857 12
11.9590432296 12
6.58095298224 3
12.1436102568 12
8.58750698932 8
9.26463604581 9
12.2338858204 12
5.90076117386 5
11.6527338558 11
10.6565120384 11
11.3703294205 11
12.1840391637 13
12.5085158151 13
11.3686131387 12
10.825234267 12
9.96435366146 13
8.12177000914 1
11.8454457296 12
10.1446489973 10
11.0674509946 12
12.7470773259 12
9.29931781419 9
10.7452000844 12
0.243671347983 1
4.69855535205 5
6.74070023397 6
12.8592343959 13
9.9778231031 10
6.16415292764 7
8.88030242663 9
10.3094886764 11
12.0055257949 12
11.6181820145 12
8.88030242663 9
9.6162584822 9
9.75986621974 12
12.0151665323 12
5.94300925105 6
11.6906268418 11
10.7

IndexError: list index out of range